In [48]:
import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import Utils.DiaSourceTools as DSTools
import lsst.afw.display.ds9 as ds9
from astropy.time import Time

In [49]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_sim/"

In [51]:
directory = DATADIR+"/output/"
butler = dafPersist.Butler(directory) 
dayDiff = butler.get("dayDiff_differenceExp", {"patch":"8,4","filter":'r', "tract":0, "date":"2005-07-07"})

In [57]:
dates = butler.queryMetadata("dayDiff_class_diaSrc", format=["date"], dataId={"filter":"r"})
print dates

['2005-06-29', '2005-07-02', '2005-07-07', '2005-07-14', '2005-07-27', '2005-08-01', '2005-08-04', '2006-01-02', '2006-01-08', '2006-01-31', '2006-02-03', '2006-02-06', '2006-02-25', '2006-03-05', '2006-04-23', '2006-04-26', '2006-04-30', '2006-05-04', '2006-05-20', '2006-05-24', '2006-05-28', '2006-06-02', '2006-06-06', '2006-06-20', '2006-06-25', '2006-06-30', '2006-07-03', '2006-07-19', '2006-07-23', '2006-07-27', '2007-01-13', '2007-01-17', '2007-01-20', '2007-01-23', '2007-02-09', '2007-02-13', '2007-02-20', '2007-02-22', '2007-03-17', '2007-03-20', '2007-03-24']


In [47]:
patches = ["8,4"]
dates = ["2005-07-07"]

multi_matches = None
for date in dates:
    date_catalog = None
    for patch in patches:
        if butler.datasetExists("dayDiff_class_diaSrc", {"patch":patch,"filter":'r', "tract":0, "date":date}) == True:
            catalog = butler.get("dayDiff_class_diaSrc", {"patch":patch,"filter":'r', "tract":0, "date":date})
            if date_catalog is None: 
                date_catalog = catalog.copy()
            else:
                date_catalog.extend(catalog)
                
    if multi_matches is None and len(date_catalog)>0:
        multi_matches = afwTable.MultiMatch(date_catalog[0].schema, {'date':float})
    
    t = Time(date)
    multi_matches.add(date_catalog, {'date':t.mjd})
                
results = multi_matches.finish(removeAmbiguous=False)  

60